<a href="https://colab.research.google.com/github/estampa/IAAC-MDEF-EI/blob/main/IAAC_3_Deep_Fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Face Image Motion Model</font></b>

Adapted from [Face_Image_Motion_Model_(Photo_2_Video)_Eng.ipynb](https://colab.research.google.com/github/tg-bomze/Face-Image-Motion-Model/blob/master/Face_Image_Motion_Model_(Photo_2_Video\)_Eng.ipynb)

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [first-order-model](https://github.com/AliaksandrSiarohin/first-order-model)

Article: [First Order Motion Model for Image Animation](https://aliaksandrsiarohin.github.io/first-order-model-website/)

Creators: **[Aliaksandr Siarohin](https://github.com/AliaksandrSiarohin), [Stéphane Lathuilière](http://stelat.eu/), [Sergey Tulyakov](http://stulyakov.com/), [Elisa Ricci](http://elisaricci.eu/) and [Nicu Sebe](http://disi.unitn.it/~sebe/).**

<b><font color="black" size="+2">Put it all together:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze) & [@JamesCullum](https://github.com/JamesCullum),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

---

To get started, click on the buttons (where the red arrow indicates) in each block in turn. After clicking, wait until the execution is complete.



In [21]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Clone the repository and install all requirements</font></b>

%cd /content
!rm -rf first_order_model
!git clone https://github.com/AliaksandrSiarohin/first-order-model first_order_model
!rm -rf sample_data
%cd first_order_model
!mkdir frames

!gdown https://drive.google.com/uc?id=1_v_xW1V52gZCZnXgh1Ap_gwA9YVIzUnS
!gdown https://drive.google.com/uc?id=1idGJFxGDgbv25ZPFCX-faloByZbPu-it
  
!pip install ffmpeg
!pip install torchvision==0.5
!pip install torch==1.4

import imageio
import numpy as np
import torch
import glob
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from IPython.display import clear_output
import io
import base64
import warnings
import os
import cv2
from google.colab import files
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")
%matplotlib inline
clear_output()

In [22]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload a square video in mp4</font></b>

#@markdown **You can crop video here: https://ezgif.com/crop-video**

#@markdown **Convert video here: https://convert-video-online.com**

#@markdown ---

#@markdown *For example, you can use this available materials: https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw*
!rm -rf video
!mkdir video
uploaded = list(files.upload().keys())

if len(uploaded) > 1:
  raise ValueError('You cannot upload more than one video at a time!')

vid = uploaded[0]
os.rename(vid, vid.replace(" ", ""))
vid = vid.replace(" ", "")
!mv -f $vid video/driving.mp4
vid = 'video/driving.mp4'

fps_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FRAME_COUNT))

clear_output()

In [23]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload one or more face photos</font></b>

!rm -rf raw_images
!mkdir raw_images
uploaded = files.upload()

i = 0
raw_photolist = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  pho = "photo-" + str(i) + "." + fn.split(".")[1]
  !mv -f $fn raw_images/$pho
  raw_photolist.append(pho)
  i += 1

clear_output()

In [24]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Transform image</font></b>

#@markdown *Attention! The image is transformed only visually. Sound from the video is not transferred. In addition, FPS decreases, so to restore it, then execute the next block.*
!rm -rf video/intermediate video/final
!mkdir -p video/intermediate video/final

print('Preparing videos and photos for transformation')
source_images = []
for photoname in raw_photolist:
  source_image = imageio.imread('raw_images/' + photoname)
  source_image = resize(source_image, (256, 256))[..., :3]
  source_images.append(source_image)

placeholder_bytes = base64.b64decode('iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mP8/x8AAwMCAO+ip1sAAAAASUVORK5CYII=')
placeholder_image = imageio.imread(placeholder_bytes, '.png')
placeholder_image = resize(placeholder_image, (256, 256))[..., :3]

driving_video = imageio.mimread(vid, memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
  fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
  ims = []
  for i in range(len(driving)):
    cols = [[placeholder_image], []]
    for sourceitem in source:
      cols[0].append(sourceitem)
    cols[1].append(driving[i])
    if generated is not None:
      for generateditem in generated:
        cols[1].append(generateditem[i])

    endcols = []
    for thiscol in cols:
      endcols.append(np.concatenate(thiscol, axis=1))
    
    im = plt.imshow(np.vstack(endcols), animated=True) # np.concatenate(cols[0], axis=1)
    plt.axis('off')
    ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
  plt.close()
  return ani

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='vox-cpk.pth.tar')

#clear_output()
print('Start the transformation')
videolist = []
predictionlist = []
i = 0
for source_img in source_images:
  videoname = 'result-' + str(i) + '.mp4'
  print('Generating ' + videoname)
  predictions = make_animation(source_img, driving_video, generator, kp_detector, relative=True)
  imageio.mimsave('video/intermediate/' + videoname, [img_as_ubyte(frame) for frame in predictions])
  videolist.append(videoname)
  predictionlist.append(predictions)
  i += 1

clear_output()
print('Videos generated. Wait a few seconds...')
HTML(display(source_images, driving_video, predictionlist).to_html5_video())

Videos generated. Wait a few seconds...


In [25]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> FPS Recovery</font></b>
fps_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FRAME_COUNT))

!rm -rf frames
!mkdir frames

play_video = True #@param {type:"boolean"}
play_html = ''

add_audio = True #@param {type:"boolean"}
if add_audio == True:
  !ffmpeg -y -i $vid -vn -ar 44100 -ac 2 -ab 192K -f mp3 sound.mp3

for videoname in videolist:
  vidcap = cv2.VideoCapture('video/intermediate/' + videoname)
  success,image = vidcap.read()
  count = 0
  success = True
  while success:
    cv2.imwrite("frames/frame%09d.jpg" % count, image)
    success,image = vidcap.read()
    count += 1

  frames = []
  img = os.listdir("frames/")
  img.sort()
  for i in img:
    frames.append(imageio.imread("frames/"+i))
  frames = np.array(frames)
  dstvid = 'video/final/' + videoname
  imageio.mimsave(dstvid, frames, fps=fps_of_video)

  !rm -rf frames
  !mkdir frames
  
  print('Assembly completed for ' + videoname)
  
  if add_audio == True:
    tmpfile = dstvid.replace('.mp4', '-audio.mp4')
    !ffmpeg -i sound.mp3 -i $dstvid $tmpfile
    !rm -rf $dstvid
    !mv -f $tmpfile $dstvid

  if play_video == True:
    video = io.open(dstvid, 'r+b').read()
    encoded = base64.b64encode(video)
    play_html = play_html + ('<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /> </video>'.format(encoded.decode('ascii')))
  
clear_output()
HTML(data=play_html)

In [26]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download final videos</font></b>
lsinfo = !ls video/final
if lsinfo.count('.mp4') == 1:
  files.download('video/final/result-0.mp4')
else:
  !zip -r ../export.zip video/final
  files.download('../export.zip')

  adding: video/final/ (stored 0%)
  adding: video/final/result-0.mp4 (deflated 5%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>